In [15]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [16]:
df = pd.read_csv('master_dataset_clean.csv').drop(["County", "State", "Unnamed: 0","FIPS"], axis=1)

In [17]:
from scipy import stats
import numpy as np
#drop all upper bound outliers
# df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]
#drop all lower bound outliers
# df = df[(np.abs(stats.zscore(df)) > -3).all(axis=1)]
#drop zero row values
# df = df.loc[(df!=0).any(axis=1)]
print(len(df))
df.describe()

3121


,live 2a1w1c,min 1a1c,pct_less_hs,pct_hs,pct_some_college,pct_college,crime_per_10k_x,unemployment_rate_2019,median_household_income_2019,Food,...,Civic,Other,Required annual income after taxes,Annual taxes,Required annual income before taxes,crime_per_10k_y,violent_crime_per_10k,hard_drugs_crime_per_10k,soft_drugs_crime_per_10k,commerce_crimes_per_10k
count,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,...,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000
mean,27.035905,8.448635,13.061038,34.174015,30.826370,21.940308,213.517325,3.927587,55656.284524,7565.166293,...,3635.282602,5123.703941,45504.658122,10753.275873,56257.874079,213.517326,40.741543,8.742535,31.582034,27.082270
std,2.146110,1.639746,6.258067,7.215851,5.200671,9.534227,195.699096,1.424564,14439.081331,517.016910,...,153.979634,117.596974,3086.739206,2364.840405,4488.883826,195.699039,39.094692,14.142338,45.326798,25.419047
min,23.450000,7.250000,1.100000,7.300000,5.200000,0.000000,0.000000,0.800000,24732.000000,7238.000000,...,3341.000000,4996.000000,42127.000000,6904.000000,49489.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.800000,7.250000,8.500000,29.700000,27.300000,15.300000,84.610000,3.000000,46214.000000,7238.000000,...,3554.000000,5070.000000,43721.000000,9373.000000,53705.000000,84.605589,16.489165,0.565004,8.348988,7.850526
50%,26.590000,7.250000,11.700000,34.600000,30.800000,19.600000,177.400000,3.700000,53289.000000,7394.000000,...,3554.000000,5144.000000,44684.000000,10784.000000,55312.000000,177.402323,35.130573,4.196127,23.396531,21.669116
75%,27.560000,9.450000,16.700000,39.100000,34.200000,26.000000,290.450000,4.600000,61878.000000,7394.000000,...,3838.000000,5144.000000,46197.000000,12202.000000,57390.000000,290.448056,56.176651,11.053761,42.785566,40.073899
max,49.450000,13.500000,73.600000,57.400000,60.600000,77.600000,2741.560000,20.900000,151806.000000,8639.000000,...,3838.000000,5514.000000,76454.000000,26406.000000,102860.000000,2741.562644,793.092910,212.765957,1210.780370,309.644670


In [18]:
X = df.drop(['crime_per_10k_x','crime_per_10k_y','violent_crime_per_10k', 'hard_drugs_crime_per_10k','soft_drugs_crime_per_10k', 'commerce_crimes_per_10k'],axis=1)
y = df['crime_per_10k_y']


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [50]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="linear", input_dim=X.shape[1]))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=100, activation="linear"))

nn_model.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 10)                180       
_________________________________________________________________
dense_36 (Dense)             (None, 100)               1100      
_________________________________________________________________
dense_37 (Dense)             (None, 1)                 101       
Total params: 1,381
Trainable params: 1,381
Non-trainable params: 0
_________________________________________________________________


In [51]:
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

In [52]:
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
74/74 [==============================] - 0s 779us/step - loss: -328.7967 - accuracy: 0.0286
Epoch 2/50
74/74 [==============================] - 0s 806us/step - loss: -1085.9148 - accuracy: 0.0197
Epoch 3/50
74/74 [==============================] - 0s 683us/step - loss: -1358.7051 - accuracy: 0.0179
Epoch 4/50
74/74 [==============================] - 0s 1ms/step - loss: -1481.0698 - accuracy: 0.0171
Epoch 5/50
74/74 [==============================] - 0s 888us/step - loss: -1781.3394 - accuracy: 0.0137
Epoch 6/50
74/74 [==============================] - 0s 724us/step - loss: -2419.4658 - accuracy: 0.0056
Epoch 7/50
74/74 [==============================] - 0s 642us/step - loss: -3048.1531 - accuracy: 0.0030
Epoch 8/50
74/74 [==============================] - 0s 587us/step - loss: -3193.1289 - accuracy: 8.5470e-04
Epoch 9/50
74/74 [==============================] - 0s 656us/step - loss: -3215.2437 - accuracy: 8.5470e-04
Epoch 10/50
74/74 [==============================] - 0s 683